# Clasificación de Imágenes en Python

## 1. Importar librerías

In [6]:
import numpy as np
import os
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import keras
from keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, LeakyReLU

## 2. Cargar las imágenes

In [8]:
dirname = os.path.join(os.getcwd(), '..', '..', 'sportimages')
imgpath = dirname + os.sep

images = []
directories = []
dircount = []
prevRoot = ''
cant = 0

print("Leyendo imágenes de", imgpath)

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant += 1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print(b, end="\r")
            if prevRoot != root:
                print(root, cant)
                prevRoot = root
                directories.append(root)
                dircount.append(cant)
                cant = 0
dircount.append(cant)

dircount = dircount[1:]
dircount[0] = dircount[0] + 1
print("Directorios leídos:", len(directories))
print("Imágenes en cada directorio:", dircount)
print("Suma total de imágenes en subdirectorios:", sum(dircount))

Leyendo imágenes de c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\americano 1
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\basket 9348
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\beisball 8823
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\boxeo 7752
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\ciclismo 7125
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\f1 7533
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\futbol 5053
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Bagnato\..\..\sportimages\golf 7617
c:\Mis Proyectos\Git\Books\Aprende_Machine_Learning-Juan_Ignacio_Ba